In [11]:
import os
os.chdir(os.path.pardir)
from dataset.dataset import Dataset
from evaluation_metrics.diversity_metrics import Topic_diversity
from evaluation_metrics.topic_significance_metrics import KL_uniform
from skopt import gp_minimize, forest_minimize, dummy_minimize
from optimization.optimizer import Optimizer
from skopt.space.space import Real, Integer, Categorical
import multiprocessing as mp
from models.TorchAvitm import TorchAvitm
from gensim.models import Word2Vec

In [2]:
# Load dataset
dataset = Dataset()
dataset.load("preprocessed_datasets/20Newsgroup_validation")

True

In [3]:
# Load model
model = TorchAvitm()

In [4]:

# Set model hyperparameters
model.hyperparameters['num_topics'] = 10
model.hyperparameters['num_epochs'] = 200

In [5]:
model.hyperparameters

{'num_epochs': 200, 'num_topics': 10}

In [6]:
model.partitioning(True)

In [7]:
output = model.train_model(dataset, model.hyperparameters)

Settings: 
               N Components: 10
               Topic Prior Mean: 0.0
               Topic Prior Variance: None
               Model Type: prodLDA
               Hidden Sizes: (100, 100)
               Activation: softplus
               Dropout: 0.2
               Learn Priors: True
               Learning Rate: 0.002
               Momentum: 0.99
               Reduce On Plateau: False
               Save Dir: None
Epoch: [1/200]	Samples: [11415/2283000]	Train Loss: 373.04224558900023	Time: 0:00:07.342809
Validation loss decreased (inf --> 373.042246).  Saving model ...
Epoch: [2/200]	Samples: [22830/2283000]	Train Loss: 354.2678920109231	Time: 0:00:04.903003
Validation loss decreased (373.042246 --> 354.267892).  Saving model ...
Epoch: [3/200]	Samples: [34245/2283000]	Train Loss: 350.248574637949	Time: 0:00:05.351000
Validation loss decreased (354.267892 --> 350.248575).  Saving model ...
Epoch: [4/200]	Samples: [45660/2283000]	Train Loss: 347.33591234806175	Time: 0:00:04

In [8]:
#define metric
td = Topic_diversity()
td.score(output)

0.91

In [9]:
# Define optimization parameters
opt_params = {}
opt_params["n_calls"] = 30
opt_params["minimizer"] = forest_minimize
opt_params["different_iteration"] = 3
opt_params["n_random_starts"] = 5
#opt_params["extra_metrics"] = [kl_uniform] # List of extra metrics
opt_params["n_jobs"] = mp.cpu_count() -1 # Enable multiprocessing
opt_params["verbose"] = True
opt_params["save_path"] = "results" #create folder if it doesn't exist 

In [12]:
# Create search space for optimization
search_space = {
    'activation': Categorical({'softplus', 'relu', 'sigmoid', 'tanh',
                               'leakyrelu', 'rrelu', 'elu', 'selu'}),
    'dropout': Real(low=0.0, high=1.0),
    'learn_priors': Integer(0, 1),
    "lr": Real(low=1e-6, high=0.1, prior='log-uniform'),
    'momentum': Real(low=0.0, high=1.0),
    'solver': Categorical({'adam', 'sgd', 'adagrad', 'adadelta', 'rmsprop'}),
    "layer_0": Categorical({100, 200, 300, 400, 500, 600, 700, 800, 900, 1000}),
    "layer_1": Categorical({0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000}),
    "layer_2": Categorical({0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000}),
}



In [14]:
# Initialize optimizer
optimizer = Optimizer(
    model,
    dataset,
    td,
    search_space,
    opt_params)

In [15]:
# Optimize
res = optimizer.optimize()

print(res.hyperparameters) # Best values for the hyperparameters
print(res.function_values) # Score of the optimized metric
print("Optimized metric: "+res.optimized_metric)

------------------------------------------
------------------------------------------
Bayesian optimization parameters:
-n_calls:  30 
-model_runs:  10 
-n_random_starts:  5 
-minimizer:  forest_minimize
-acq_func:  LCB
------------------------------------------
Iteration No: 1 started. Evaluating function at random point.
Settings: 
               N Components: 10
               Topic Prior Mean: 0.0
               Topic Prior Variance: None
               Model Type: prodLDA
               Hidden Sizes: (200,)
               Activation: softplus
               Dropout: 0.9847997823068501
               Learn Priors: 0
               Learning Rate: 0.016437678289695102
               Momentum: 0.7964055405131697
               Reduce On Plateau: False
               Save Dir: None
Epoch: [1/200]	Samples: [11415/2283000]	Train Loss: 371.8197479502026	Time: 0:00:04.041998
Validation loss decreased (inf --> 371.819748).  Saving model ...
Epoch: [2/200]	Samples: [22830/2283000]	Train Loss

KeyboardInterrupt: 